# References

## Data Source

https://www.kaggle.com/datasets/jihyeseo/online-retail-data-set-from-uci-ml-repo

# Cleaning and Segmenting

In [1]:
# importing packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
sns.set_style('darkgrid')
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
# loading retail data
df = pd.read_excel('C:/Users/KodavaliPavanKumar/Desktop/Training/github_folders/Projects/MM_Sample/Data/Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df.duplicated().sum()

5268

In [4]:
# removing duplicates
df = df[~df.duplicated()]
df.shape

(536641, 8)

In [5]:
# removing all the invoice number who starts with 'C' as they are returned orders
df = df[df['InvoiceNo'].str.startswith('C')!=True]
df.shape

(527390, 8)

In [6]:
# keeping only those transactions that have successfully ordered
df = df[df['Quantity']>=0]
df.shape

(526054, 8)

In [7]:
# putting UK as one country and combine rest countries into one category
df['Country'] = df['Country'].apply(lambda x:'United Kingdom' if x=='United Kingdom' else 'Others')
df.Country.value_counts(normalize=True)

United Kingdom    0.914627
Others            0.085373
Name: Country, dtype: float64

In [8]:
# removing all the above entries
df = df[df['Description'].str.startswith('?')!=True]
df.shape

(526048, 8)

In [9]:
# checking the data where description = * and it is noted that customerid is NaN
df[df['Description'].str.startswith('*')==True]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
20749,538071,21120,*Boombox Ipod Classic,1,2010-12-09 14:09:00,16.98,NaN,United Kingdom
35675,539437,20954,*USB Office Mirror Ball,1,2010-12-17 14:54:00,8.47,NaN,United Kingdom
37095,539453,20954,*USB Office Mirror Ball,1,2010-12-17 17:08:00,8.47,NaN,United Kingdom


In [10]:
# replacing with appropriate name
df['Description'] = df['Description'].replace(('*Boombox Ipod Classic','*USB Office Mirror Ball'),
                                             ('BOOMBOX IPOD CLASSIC','USB OFFICE MIRROR BALL'))

In [11]:
# Description have actual entries in uppercase words and those who don't have are some of the noises in the dataset
df[df['Description'].str.islower()==True]['Description'].value_counts()

check                                  39
found                                  25
adjustment                             14
amazon                                  8
had been put aside                      5
dotcom                                  4
mailout                                 3
returned                                2
taig adjust                             2
test                                    2
found box                               1
alan hodge cant mamage this section     1
wrongly coded 20713                     1
website fixed                           1
allocate stock for dotcom orders ta     1
michel oops                             1
wrongly coded 23343                     1
check?                                  1
add stock to allocate online orders     1
dotcomstock                             1
wrongly sold (22719) barcode            1
to push order througha s stock was      1
came coded as 20713                     1
rcvd be air temp fix for dotcom si

In [12]:
# removing all the above noises
df = df[df['Description'].str.islower()!=True]
df.shape

(525920, 8)

In [13]:
# Description have actual entries in uppercase words and those who don't have are some of the noises in the dataset
df[df['Description'].str.istitle()==True]['Description'].value_counts()

Manual                                 323
Next Day Carriage                       79
Bank Charges                            12
Dotcomgiftshop Gift Voucher £20.00       9
Dotcomgiftshop Gift Voucher £10.00       8
Found                                    8
Dotcomgiftshop Gift Voucher £30.00       7
Amazon                                   7
Dotcomgiftshop Gift Voucher £50.00       4
High Resolution Image                    3
Dotcomgiftshop Gift Voucher £40.00       3
Adjustment                               2
John Lewis                               1
Dotcomgiftshop Gift Voucher £100.00      1
Amazon Adjustment                        1
Name: Description, dtype: int64

In [14]:
# removing all the above listed noises
df = df[df['Description'].str.istitle()!=True]
df.shape

(525452, 8)

In [15]:
df['Description'] = df['Description'].str.strip()

In [16]:
# removing entries where customer id is null
df = df[~df.CustomerID.isnull()]
df.shape

(392353, 8)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392353 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392353 non-null  object        
 1   StockCode    392353 non-null  object        
 2   Description  392353 non-null  object        
 3   Quantity     392353 non-null  int64         
 4   InvoiceDate  392353 non-null  datetime64[ns]
 5   UnitPrice    392353 non-null  float64       
 6   CustomerID   392353 non-null  float64       
 7   Country      392353 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 26.9+ MB


In [18]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [19]:
# checking random 5 rows from data
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
341074,566733,23307,SET OF 60 PANTRY DESIGN CAKE CASES,24,2011-09-14 13:54:00,0.55,15103.0,United Kingdom
456350,575687,23188,VINTAGE 2 METRE FOLDING RULER,36,2011-11-10 16:03:00,1.65,16126.0,United Kingdom
142118,548552,21936,RED RETROSPOT PICNIC BAG,5,2011-04-01 09:06:00,2.95,17613.0,United Kingdom
85030,543458,22698,PINK REGENCY TEACUP AND SAUCER,2,2011-02-08 13:28:00,2.95,14587.0,United Kingdom
14015,537441,21876,POTTERING MUG,6,2010-12-06 17:26:00,1.25,13564.0,United Kingdom


# Creating Intermediate Variables

In [20]:
# creating some columns for exploratory
df['Amount'] = df['Quantity']*df['UnitPrice']

In [22]:
df.shape

(392353, 9)

In [23]:
# getting the data from timestamp
df_c = df.copy()
df_c['date'] = df_c['InvoiceDate'].dt.strftime('%Y-%m-%d')
df_c.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01


# Segmenting data based on dates

## Transactions of 2010-12-01

In [25]:
# segmenting the transaction for the date '2010-12-01'
df_2010_12_01 = df_c.loc[df_c['date'] == '2010-12-01']
df_2010_12_01.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12-01


In [26]:
df_2010_12_01.shape

(1896, 10)

In [27]:
# slicing the 2010-12-01 transactions for necessary features
df_2010_12_01_final = df_2010_12_01[['CustomerID','date','Quantity','UnitPrice','Amount']]
df_2010_12_01_final.head()

,CustomerID,date,Quantity,UnitPrice,Amount
0,17850.0,2010-12-01,6,2.55,15.30
1,17850.0,2010-12-01,6,3.39,20.34
2,17850.0,2010-12-01,8,2.75,22.00
3,17850.0,2010-12-01,6,3.39,20.34
4,17850.0,2010-12-01,6,3.39,20.34


In [41]:
# creating frequency column - number of item a customer purchased 
df_2010_12_01_f = pd.DataFrame(df_2010_12_01_final.groupby(['CustomerID'])['date'].count().reset_index())
df_2010_12_01_f.columns = ['CustomerID','frequency']
df_2010_12_01_f.tail()

,CustomerID,frequency
90,18011.0,28
91,18074.0,13
92,18085.0,9
93,18144.0,3
94,18229.0,7


In [42]:
# grouping the data based o customerID
df_2010_12_01_m = pd.DataFrame(df_2010_12_01_final.groupby(['CustomerID'])[['Amount','Quantity','UnitPrice']].sum().reset_index())
df_2010_12_01_m.head()

,CustomerID,Amount,Quantity,UnitPrice
0,12431.0,358.25,107,73.90
1,12433.0,1919.14,1852,102.67
2,12583.0,855.86,449,55.29
3,12662.0,261.48,157,44.37
4,12748.0,4.95,1,4.95


In [43]:
print(df_2010_12_01_m.shape, df_2010_12_01_f.shape)

(95, 4) (95, 2)


In [44]:
# merging frequency and monetary tables to final table
df_2010_12_01_fm = pd.merge(df_2010_12_01_m, df_2010_12_01_f, on='CustomerID',how='inner')
df_2010_12_01_fm.head()

,CustomerID,Amount,Quantity,UnitPrice,frequency
0,12431.0,358.25,107,73.90,14
1,12433.0,1919.14,1852,102.67,73
2,12583.0,855.86,449,55.29,20
3,12662.0,261.48,157,44.37,15
4,12748.0,4.95,1,4.95,1


In [46]:
df_2010_12_01_fm.shape

(95, 5)

In [47]:
df_2010_12_01_fm.isnull().sum()

CustomerID    0
Amount        0
Quantity      0
UnitPrice     0
frequency     0
dtype: int64

## Transactions of 2010-12-02

In [48]:
# segmenting the transaction for the date '2010-12-02'
df_2010_12_02 = df_c.loc[df_c['date'] == '2010-12-02']
df_2010_12_02.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount,date
3108,536598,21421,PORCELAIN ROSE LARGE,12,2010-12-02 07:48:00,1.25,13090.0,United Kingdom,15.0,2010-12-02
3109,536598,21422,PORCELAIN ROSE SMALL,16,2010-12-02 07:48:00,0.85,13090.0,United Kingdom,13.6,2010-12-02
3110,536598,22178,VICTORIAN GLASS HANGING T-LIGHT,24,2010-12-02 07:48:00,1.25,13090.0,United Kingdom,30.0,2010-12-02
3111,536598,22617,BAKING SET SPACEBOY DESIGN,24,2010-12-02 07:48:00,4.25,13090.0,United Kingdom,102.0,2010-12-02
3112,536599,22968,ROSE COTTAGE KEEPSAKE BOX,8,2010-12-02 07:49:00,8.50,15694.0,United Kingdom,68.0,2010-12-02


In [49]:
# slicing the 2010-12-02 transactions for necessary features
df_2010_12_02_final = df_2010_12_02[['CustomerID','date','Quantity','UnitPrice','Amount']]
df_2010_12_02_final.head()

,CustomerID,date,Quantity,UnitPrice,Amount
3108,13090.0,2010-12-02,12,1.25,15.0
3109,13090.0,2010-12-02,16,0.85,13.6
3110,13090.0,2010-12-02,24,1.25,30.0
3111,13090.0,2010-12-02,24,4.25,102.0
3112,15694.0,2010-12-02,8,8.50,68.0


In [52]:
df_2010_12_02_final.shape

(1958, 5)

In [51]:
# creating frequency column - number of item a customer purchased 
df_2010_12_02_f = pd.DataFrame(df_2010_12_02_final.groupby(['CustomerID'])['date'].count().reset_index())
df_2010_12_02_f.columns = ['CustomerID','frequency']
df_2010_12_02_f.tail()

,CustomerID,frequency
93,17964.0,35
94,17976.0,57
95,18041.0,73
96,18168.0,27
97,18239.0,29


In [53]:
# grouping the data based o customerID
df_2010_12_02_m = pd.DataFrame(df_2010_12_02_final.groupby(['CustomerID'])[['Amount','Quantity','UnitPrice']].sum().reset_index())
df_2010_12_02_m.head()

,CustomerID,Amount,Quantity,UnitPrice
0,12738.0,155.35,148,43.45
1,12748.0,4.25,1,4.25
2,12855.0,38.10,30,4.65
3,12915.0,199.65,41,100.00
4,12971.0,45.12,84,2.92


In [54]:
# merging frequency and monetary tables to final table
df_2010_12_02_fm = pd.merge(df_2010_12_02_m, df_2010_12_02_f, on='CustomerID',how='inner')
df_2010_12_02_fm.head()

,CustomerID,Amount,Quantity,UnitPrice,frequency
0,12738.0,155.35,148,43.45,11
1,12748.0,4.25,1,4.25,1
2,12855.0,38.10,30,4.65,3
3,12915.0,199.65,41,100.00,13
4,12971.0,45.12,84,2.92,5


In [55]:
df_2010_12_02_fm.shape

(98, 5)

In [56]:
df_2010_12_02_fm.isnull().sum()

CustomerID    0
Amount        0
Quantity      0
UnitPrice     0
frequency     0
dtype: int64

# Saving transaction segments

In [72]:
df_2010_12_01_fm.head()

,CustomerID,Amount,Quantity,frequency
0,12431.0,358.25,107,14
1,12433.0,1919.14,1852,73
2,12583.0,855.86,449,20
3,12662.0,261.48,157,15
4,12748.0,4.95,1,1


In [64]:
df_2010_12_01_fm = df_2010_12_01_fm.drop('UnitPrice', 1)

In [65]:
df_2010_12_01_fm.shape

(95, 4)

In [66]:
# saving 2010-12-01 customers data
df_2010_12_01_fm.to_csv(r'C:/Users/KodavaliPavanKumar/Desktop/Training/github_folders/Projects/MM_Sample/Data/df_2010_12_01_fm.csv', index=False)

In [71]:
df_2010_12_02_fm.head()

,CustomerID,Amount,Quantity,frequency
0,12738.0,155.35,148,11
1,12748.0,4.25,1,1
2,12855.0,38.10,30,3
3,12915.0,199.65,41,13
4,12971.0,45.12,84,5


In [68]:
df_2010_12_02_fm = df_2010_12_02_fm.drop('UnitPrice', 1)

In [69]:
df_2010_12_02_fm.shape

(98, 4)

In [70]:
# saving 2010-12-02 customers data
df_2010_12_02_fm.to_csv(r'C:/Users/KodavaliPavanKumar/Desktop/Training/github_folders/Projects/MM_Sample/Data/df_2010_12_02_fm.csv', index=False)

#### Note: Model to predict the Amount based on Quantity and Frequency is in 'predicting_amount.ipynb' file